- 2020-08-22
- OS: Ubuntu 18.04
- TF: 2.2.0
- yolov4: 1.2.0
- edgetpu_compiler: 14.1.317412892

In [1]:
from yolov4.tf import YOLOv4

yolo = YOLOv4(tiny=True, tpu=True)

yolo.classes = "dataset/coco.names"

yolo.make_model(activation1="relu")
yolo.load_weights("yolov4-tiny-relu.weights", weights_type="yolo")

dataset = yolo.load_dataset(
    "dataset/train2017.txt",
    training=False,
    image_path_prefix=r"D:\coco\train2017"
)

yolo.save_as_tflite(
    "yolov4-tiny-relu.tflite",
    quantization="full_int8",
    data_set=dataset,
    num_calibration_steps=500
)

In [2]:
!edgetpu_compiler yolov4-tiny-relu.tflite

Edge TPU Compiler version 14.1.317412892

Model compiled successfully in 1094 ms.

Input model: test/yolov4-tiny-relu.tflite
Input size: 5.96MiB
Output model: yolov4-tiny-relu_edgetpu.tflite
Output size: 6.19MiB
On-chip memory used for caching model parameters: 5.92MiB
On-chip memory remaining for caching model parameters: 208.25KiB
Off-chip memory used for streaming uncached model parameters: 0.00B
Number of Edge TPU subgraphs: 1
Total number of operations: 149
Operation log: yolov4-tiny-relu_edgetpu.log

Model successfully compiled but not all operations are supported by the Edge TPU. A percentage of the model will instead run on the CPU, which is slower. If possible, consider updating your model to use only operations supported by the Edge TPU. For details, visit g.co/coral/model-reqs.
Number of operations that will run on Edge TPU: 50
Number of operations that will run on CPU: 99
See the operation log file for individual operation details.


In [3]:
!cat yolov4-tiny-relu_edgetpu.log

Edge TPU Compiler version 14.1.317412892
Input: test/yolov4-tiny-relu.tflite
Output: yolov4-tiny-relu_edgetpu.tflite

Operator                       Count      Status

SPLIT_V                        12         Operation is working on an unsupported data type
MUL                            18         More than one subgraph is not supported
SUB                            6          More than one subgraph is not supported
LOGISTIC                       2          Mapped to Edge TPU
SPLIT                          7          Mapped to Edge TPU
CONCATENATION                  7          Mapped to Edge TPU
CONCATENATION                  2          More than one subgraph is not supported
RESIZE_BILINEAR                1          Mapped to Edge TPU
DEQUANTIZE                     14         Operation is working on an unsupported data type
CONV_2D                        21         Mapped to Edge TPU
QUANTIZE                       7          Mapped to Edge TPU
QUANTIZE                       16     

- 2020-08-22
- HW: Coral Dev Board
- OS: Mendel
- tflite_runtime: 2.1.0.post1
- yolov4: 1.2.0

In [ ]:
from yolov4.tflite import YOLOv4
import cv2

yolo = YOLOv4(tiny=True, tpu=True)

yolo.classes = "coco.names"

yolo.load_tflite("yolov4-tiny-relu_edgetpu.tflite")

yolo.inference(
    "/dev/video1",
    is_image=False,
    cv_apiPreference=cv2.CAP_V4L2,
    cv_frame_size=(640, 480),
    cv_fourcc="YUYV"
)